In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from pandas import read_csv
from pandas import DataFrame

from numpy import dstack

import copy

import os
from glob import glob

from path import Path

from numpy import mean
from numpy import std

from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Flatten
from keras.layers import Dropout
from keras.layers import LSTM
from keras.layers import SimpleRNN

from keras.models import model_from_json

from keras.layers import Bidirectional

from keras.utils.vis_utils import plot_model

from keras.callbacks import EarlyStopping

from keras.optimizers import Adam

import keras.backend as K

import random

from sklearn.model_selection import train_test_split

from matplotlib import pyplot

Using TensorFlow backend.


In [2]:
import tensorflow as tf

tf.config.list_physical_devices('GPU')

[PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]

## Load Data

In [3]:
%%time
from helpers.classes.data_loader import DataLoader

root_path = "/home/ubuntu/pose-estimation-jupyter/augmented-keypoints-339"

sample_dir_names = [n for n in os.listdir(root_path) if os.path.isdir(f"{root_path}/{n}")]

samples = DataLoader.get_samples_list(sample_dir_names, root_path)

y_labels = DataLoader.get_y_labels(sample_dir_names) # classifier labels, where 0 = backflip and 1 = flack

Loading frames for 0/339
Loading frames for 1/339
Loading frames for 2/339
Loading frames for 3/339
Loading frames for 4/339
Loading frames for 5/339
Loading frames for 6/339
Loading frames for 7/339
Loading frames for 8/339
Loading frames for 9/339
Loading frames for 10/339
Loading frames for 11/339
Loading frames for 12/339
Loading frames for 13/339
Loading frames for 14/339
Loading frames for 15/339
Loading frames for 16/339
Loading frames for 17/339
Loading frames for 18/339
Loading frames for 19/339
Loading frames for 20/339
Loading frames for 21/339
Loading frames for 22/339
Loading frames for 23/339
Loading frames for 24/339
Loading frames for 25/339
Loading frames for 26/339
Loading frames for 27/339
Loading frames for 28/339
Loading frames for 29/339
Loading frames for 30/339
Loading frames for 31/339
Loading frames for 32/339
Loading frames for 33/339
Loading frames for 34/339
Loading frames for 35/339
Loading frames for 36/339
Loading frames for 37/339
Loading frames for 38/

Loading frames for 308/339
Loading frames for 309/339
Loading frames for 310/339
Loading frames for 311/339
Loading frames for 312/339
Loading frames for 313/339
Loading frames for 314/339
Loading frames for 315/339
Loading frames for 316/339
Loading frames for 317/339
Loading frames for 318/339
Loading frames for 319/339
Loading frames for 320/339
Loading frames for 321/339
Loading frames for 322/339
Loading frames for 323/339
Loading frames for 324/339
Loading frames for 325/339
Loading frames for 326/339
Loading frames for 327/339
Loading frames for 328/339
Loading frames for 329/339
Loading frames for 330/339
Loading frames for 331/339
Loading frames for 332/339
Loading frames for 333/339
Loading frames for 334/339
Loading frames for 335/339
Loading frames for 336/339
Loading frames for 337/339
Loading frames for 338/339
CPU times: user 1min 8s, sys: 832 ms, total: 1min 9s
Wall time: 1min 9s


In [4]:
i = 5

# Print some sample dir names
a = [print(sample_dir_name) for sdn_i, sample_dir_name in enumerate(sample_dir_names) if sdn_i < i]
a = [print(y_label) for y_i, y_label in enumerate(y_labels) if y_i < i]

backflip-33-mario-aug-2
backflip-46-dagne-aug-1
backflip-63-allar-aug-1
backflip-47-dagne-aug-1
flack-56-martin-aug-0
0
0
0
0
1


## Data padding

In [5]:
%%time
from helpers.classes.padder import Padder

padded_samples_list = Padder.get_padded_samples(samples)
categorical_y_labels = DataLoader.get_categorical_y_labels(y_labels)

padded_samples_ndarray = np.asarray(padded_samples_list)
categorical_y_labels_ndarray = np.asarray(categorical_y_labels)

y_labels_stacked shape: (1, 1, 339)
y_labels_categorical shape: (1, 1, 339, 2)
y_labels_squeezed shape (339, 2)
y_label categorical: [1.0, 0.0]
y_label categorical: [1.0, 0.0]
y_label categorical: [1.0, 0.0]
y_label categorical: [1.0, 0.0]
y_label categorical: [0.0, 1.0]
CPU times: user 3.32 s, sys: 7.78 ms, total: 3.33 s
Wall time: 3.32 s


## Model loading

In [6]:
# load json and create model
json_file = open('model-output-simple-rnn-339/model-0.json', 'r')
loaded_model_json = json_file.read()
json_file.close()
loaded_model = model_from_json(loaded_model_json)
# load weights into new model
loaded_model.load_weights("model-output-simple-rnn-339/model-0.h5")
print("Loaded model from disk")

Loaded model from disk


## Visualizing gradients

(110, 30)
[1. 0.]


AttributeError: 'NoneType' object has no attribute 'get_gradients'

## Sample

In [42]:
example_sample_idx = 139
model_history_idx = 1

example_sample_dir_name = sample_dir_names[example_sample_idx]
example_sample = padded_samples_ndarray[example_sample_idx]
example_model = loaded_model

## Predictions

In [43]:
def predictions(loaded_model, samples, sample_dir_names, y_labels):
    ynew = loaded_model.predict_classes(samples)
    # show the inputs and predicted outputs
    for i in range(len(samples)):
        pred_y = ynew[i]
        actual_y = y_labels[i]
    
        same = False
        if pred_y == actual_y:
            same = True
    
        print("Name=%s, X=%s, Predicted=%s, Actual=%s, same=%s" % (sample_dir_names[i], i, pred_y, actual_y, same))
    
predictions(example_model, padded_samples_ndarray, sample_dir_names, y_labels)

Name=backflip-51-rasmus-aug-3, X=0, Predicted=1, Actual=0, same=False
Name=flack-9-hendrik-aug-40, X=1, Predicted=1, Actual=1, same=True
Name=flack-49-kristiin-aug-30, X=2, Predicted=1, Actual=1, same=True
Name=backflip-52-rasmus-aug-44, X=3, Predicted=1, Actual=0, same=False
Name=flack-9-hendrik-aug-14, X=4, Predicted=1, Actual=1, same=True
Name=backflip-46-dagne-aug-18, X=5, Predicted=0, Actual=0, same=True
Name=backflip-31-hendrik-aug-83, X=6, Predicted=0, Actual=0, same=True
Name=backflip-56-margus-aug-41, X=7, Predicted=0, Actual=0, same=True
Name=backflip-53-rasmus-aug-52, X=8, Predicted=0, Actual=0, same=True
Name=flack-67-rasmus-aug-28, X=9, Predicted=1, Actual=1, same=True
Name=flack-30-rasmus-aug-76, X=10, Predicted=1, Actual=1, same=True
Name=flack-30-rasmus-aug-44, X=11, Predicted=1, Actual=1, same=True
Name=backflip-37-mario-aug-25, X=12, Predicted=0, Actual=0, same=True
Name=backflip-31-hendrik-aug-93, X=13, Predicted=0, Actual=0, same=True
Name=backflip-52-rasmus-aug-97,

Name=flack-80-martin-aug-23, X=2410, Predicted=1, Actual=1, same=True
Name=backflip-34-mario-aug-7, X=2411, Predicted=0, Actual=0, same=True
Name=backflip-57-margus-aug-48, X=2412, Predicted=0, Actual=0, same=True
Name=flack-5-martin-aug-96, X=2413, Predicted=1, Actual=1, same=True
Name=backflip-20-martin-aug-73, X=2414, Predicted=0, Actual=0, same=True
Name=flack-30-rasmus-aug-64, X=2415, Predicted=1, Actual=1, same=True
Name=flack-30-rasmus-aug-58, X=2416, Predicted=1, Actual=1, same=True
Name=flack-44-kristiin-aug-5, X=2417, Predicted=1, Actual=1, same=True
Name=flack-79-martin-aug-9, X=2418, Predicted=1, Actual=1, same=True
Name=backflip-57-margus-aug-54, X=2419, Predicted=0, Actual=0, same=True
Name=flack-4-martin-aug-87, X=2420, Predicted=1, Actual=1, same=True
Name=backflip-10-hendrik-aug-63, X=2421, Predicted=0, Actual=0, same=True
Name=flack-57-martin-aug-41, X=2422, Predicted=1, Actual=1, same=True
Name=backflip-51-rasmus-aug-27, X=2423, Predicted=1, Actual=0, same=False
Name

Name=backflip-8-rasmus-aug-45, X=4420, Predicted=0, Actual=0, same=True
Name=flack-3-martin-aug-59, X=4421, Predicted=1, Actual=1, same=True
Name=flack-56-martin-aug-9, X=4422, Predicted=1, Actual=1, same=True
Name=flack-6-hendrik-aug-72, X=4423, Predicted=1, Actual=1, same=True
Name=backflip-26-tiit-aug-44, X=4424, Predicted=0, Actual=0, same=True
Name=backflip-47-dagne-aug-83, X=4425, Predicted=0, Actual=0, same=True
Name=flack-56-martin-aug-84, X=4426, Predicted=1, Actual=1, same=True
Name=flack-5-martin-aug-69, X=4427, Predicted=1, Actual=1, same=True
Name=backflip-43-kristjan-aug-10, X=4428, Predicted=0, Actual=0, same=True
Name=flack-15-rasmus-aug-35, X=4429, Predicted=1, Actual=1, same=True
Name=flack-29-julia-aug-42, X=4430, Predicted=1, Actual=1, same=True
Name=flack-32-rasmus-aug-49, X=4431, Predicted=1, Actual=1, same=True
Name=flack-21-allar-aug-27, X=4432, Predicted=1, Actual=1, same=True
Name=flack-29-julia-aug-50, X=4433, Predicted=1, Actual=1, same=True
Name=flack-16-ra

Name=flack-59-martin-aug-48, X=6886, Predicted=1, Actual=1, same=True
Name=flack-65-rasmus-aug-31, X=6887, Predicted=1, Actual=1, same=True
Name=flack-69-rasmus-aug-64, X=6888, Predicted=1, Actual=1, same=True
Name=backflip-34-mario-aug-19, X=6889, Predicted=0, Actual=0, same=True
Name=backflip-20-martin-aug-70, X=6890, Predicted=0, Actual=0, same=True
Name=backflip-59-margus-aug-8, X=6891, Predicted=0, Actual=0, same=True
Name=backflip-63-allar-aug-54, X=6892, Predicted=0, Actual=0, same=True
Name=flack-26-margus-aug-67, X=6893, Predicted=1, Actual=1, same=True
Name=backflip-10-hendrik-aug-8, X=6894, Predicted=0, Actual=0, same=True
Name=backflip-39-margus-aug-94, X=6895, Predicted=0, Actual=0, same=True
Name=backflip-2-allar-aug-89, X=6896, Predicted=0, Actual=0, same=True
Name=backflip-7-rasmus-aug-16, X=6897, Predicted=1, Actual=0, same=False
Name=flack-78-martin-aug-58, X=6898, Predicted=1, Actual=1, same=True
Name=flack-10-rasmus-aug-20, X=6899, Predicted=1, Actual=1, same=True
N

Name=backflip-47-dagne-aug-7, X=9639, Predicted=0, Actual=0, same=True
Name=flack-66-rasmus-aug-18, X=9640, Predicted=1, Actual=1, same=True
Name=backflip-1-allar-aug-83, X=9641, Predicted=0, Actual=0, same=True
Name=flack-59-martin-aug-9, X=9642, Predicted=1, Actual=1, same=True
Name=flack-10-rasmus-aug-18, X=9643, Predicted=1, Actual=1, same=True
Name=backflip-37-mario-aug-19, X=9644, Predicted=0, Actual=0, same=True
Name=flack-24-belinda-aug-72, X=9645, Predicted=1, Actual=1, same=True
Name=backflip-5-rasmus-aug-10, X=9646, Predicted=0, Actual=0, same=True
Name=backflip-40-margus-aug-55, X=9647, Predicted=0, Actual=0, same=True
Name=flack-10-rasmus-aug-43, X=9648, Predicted=1, Actual=1, same=True
Name=backflip-48-joosep-aug-28, X=9649, Predicted=0, Actual=0, same=True
Name=flack-3-martin-aug-0, X=9650, Predicted=1, Actual=1, same=True
Name=backflip-33-mario-aug-85, X=9651, Predicted=0, Actual=0, same=True
Name=flack-39-margus-aug-30, X=9652, Predicted=1, Actual=1, same=True
Name=fla

## Activation

In [48]:
layer_names = [layer.name for layer in example_model.layers]

print(example_sample_dir_name)
print(example_sample.shape)
print(layer_names)

def get_activations(model, example_sample): 
    n_timesteps, n_features = example_sample.shape[0], example_sample.shape[1]
    
    x = np.zeros((1, n_timesteps, n_features))
    
    for t, timestep in enumerate(example_sample):
        for f, feature in enumerate(timestep):
            x[0, t, f] = example_sample[t][f]
                
    output = model.get_layer('simple_rnn_1').output
    
    f = K.function([model.input], [output])
    
    return f([x])[0][0]

activations = get_activations(example_model, example_sample)
#print(actications)
#act.shape

flack-7-hendrik-aug-39
(110, 30)
['simple_rnn_1', 'dropout_1', 'dense_1', 'flatten_1', 'dense_2']
n_timesteps: 110, n_features: 30


In [50]:
from io import BytesIO
from PIL import Image as PILImage
from PIL import ImageDraw
from IPython.display import Image

def get_image(img, n_timesteps, img_idx, cell_size=48):
    img_width = n_timesteps * 25
    cell_size = int(img_width / n_timesteps)
    
    pil_image = PILImage.fromarray(img.astype(np.uint8))
    
    resized_pil_image = pil_image.resize((img_width, cell_size))
    
    draw = ImageDraw.Draw(resized_pil_image)
    
    for n_timestep in range(n_timesteps):
        text = str((img_idx * 30) + n_timestep)
        xy = (n_timestep * cell_size, 0)
        
        draw.text(xy, text)
        
    f = BytesIO()
    resized_pil_image.save(f, 'png')
    return Image(data=f.getvalue())

def visualize_neurons(act, cell_size=48):
    n_neurons = act.shape[1]
    n_timesteps = act.shape[0]
    
    fill_value = 128
    
    img = np.full((n_neurons + 1, n_timesteps, 3), fill_value)
    
    # add 1 to each value in matrix and then divide by 2
    scores = (act[:, :].T + 1) / 2
    
    img[1:, :, 0] = 255 * (1 - scores)
    img[1:, :, 1] = 255 * scores

    first_hs_img = img[:, :30, :]
    second_hs_img = img[:, 30:60, :]
    third_hs_img = img[:, 60:90, :]
    fourth_hs_img = img[:, 90:, :]
    
    imgs = [first_hs_img,
            second_hs_img,
            third_hs_img,
            fourth_hs_img]
    
    actual_imgs = []
    for i, img in enumerate(imgs):
        n_img_timesteps = img.shape[1]
        
        actual_imgs.append(get_image(img, n_img_timesteps, i))
    
    return actual_imgs

example_sample_imgs = visualize_neurons(activations)

for img in example_sample_imgs:
    display(img)